In [2]:
import os
import glob
import pandas as pd

In [4]:
input_fl = r"C:\Users\Asus\Desktop\nifty50_csv"
output_fl = r"C:\Users\Asus\Desktop\nifty50_output"
os.makedirs(output_fl, exist_ok = True)

In [5]:
print("Input Folder", input_fl)
print("Output Folder", output_fl)

Input Folder C:\Users\Asus\Desktop\nifty50_csv
Output Folder C:\Users\Asus\Desktop\nifty50_output


In [6]:
files = sorted(glob.glob(os.path.join(input_fl, "*.csv")))
print(f"Found {len(files)} CSV files.")
print("Example filenames:", files[:5])

Found 52 CSV files.
Example filenames: ['C:\\Users\\Asus\\Desktop\\nifty50_csv\\ADANIPORTS.csv', 'C:\\Users\\Asus\\Desktop\\nifty50_csv\\ASIANPAINT.csv', 'C:\\Users\\Asus\\Desktop\\nifty50_csv\\AXISBANK.csv', 'C:\\Users\\Asus\\Desktop\\nifty50_csv\\BAJAJ-AUTO.csv', 'C:\\Users\\Asus\\Desktop\\nifty50_csv\\BAJAJFINSV.csv']


In [30]:
#L &C
dfs = []
failed_files = []

for f in files:
    try:
        df = pd.read_csv(f, low_memory = False)
    except Exception as e:
        print(f"Failed to read {os.path.basename(f)}: {e}")
        failed_files.append(f)
        continue
        
    if 'Symbol' not in df.columns:
        ticker = os.path.splitext(os.path.basename(f))[0]
        df['Symbol'] = ticker
        
    df['Symbol'] = df['Symbol'].astype(str).str.strip()
    
    if 'Date' not in df.columns:
        possible = [c for c in df.columns if c.lower().strip() == 'date']
        
        if possible:
            df = df.rename(columns = {possible[0]: 'Date'})
        else:
            print(f"Skipped {os.path.basename(f)} - no Date column found. ")
            failed_files.append(f)
            continue
            
    dfs.append(df)
    
df_all = pd.concat(dfs, ignore_index = True)
print("\n Combined DataFrame created.")
print("Shape:", df_all.shape)
print("Failed files:", len(failed_files))
if failed_files:
    print("Failed file names:", [os.path.basename(x) for x in failed_files])

Skipped stock_metadata.csv - no Date column found. 

 Combined DataFrame created.
Shape: (470384, 15)
Failed files: 1
Failed file names: ['stock_metadata.csv']


In [8]:
print("\n Preview (first 10 rows):")
display(df_all.head(10))

print("\n Column list:")
print(list(df_all.columns))

distinct_tickers = df_all['Symbol'].nunique()
print(f"Total distinct tickers: {distinct_tickers}")

missing_date = df_all['Date'].isna().sum()
missing_close = df_all['Close'].isna().sum() if 'Close' in df_all.columns else 0
total_rows = len(df_all)

print(f"\n Total rows: {total_rows}")
print(f"Rows with missing Date: {missing_date}")
print(f"Rows with missing Close: {missing_close}")

print("\n Top 10 tickers by row count: ")
display(df_all['Symbol'].value_counts().head(10))

preview_path = os.path.join(output_fl, "nifty50_combined_preview.csv")
df_all.head(500).to_csv(preview_path, index=False)
print(f"\n Saved first 500 rows preview -> {preview_path}")


 Preview (first 10 rows):


,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.00,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.00,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.00,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.00,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.00,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741
5,2007-12-04,MUNDRAPORT,EQ,969.30,985.00,1056.00,976.0,1049.00,1041.45,1015.39,4849250,4.923867e+14,NaN,1537667.0,0.3171
6,2007-12-05,MUNDRAPORT,EQ,1041.45,1061.00,1099.50,1050.0,1084.00,1082.45,1082.79,2848209,3.084010e+14,NaN,904260.0,0.3175
7,2007-12-06,MUNDRAPORT,EQ,1082.45,1089.00,1109.70,1051.0,1090.10,1081.30,1087.03,1749516,1.901771e+14,NaN,825691.0,0.4720
8,2007-12-07,MUNDRAPORT,EQ,1081.30,1100.00,1134.00,1078.0,1100.00,1102.40,1106.57,2247904,2.487465e+14,NaN,697763.0,0.3104
9,2007-12-10,MUNDRAPORT,EQ,1102.40,1110.00,1110.00,1061.1,1073.55,1075.40,1080.38,1012350,1.093727e+14,NaN,417514.0,0.4124



 Column list:
['Date', 'Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last', 'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble']
Total distinct tickers: 65

 Total rows: 470384
Rows with missing Date: 0
Rows with missing Close: 0

 Top 10 tickers by row count: 


IOC          10612
CIPLA        10612
ICICIBANK    10612
M&M          10612
ONGC         10612
RELIANCE     10612
SBIN         10612
HDFCBANK     10612
HDFC         10612
GRASIM       10612
Name: Symbol, dtype: int64


 Saved first 500 rows preview -> C:\Users\Asus\Desktop\nifty50_output\nifty50_combined_preview.csv


In [16]:
#c
from IPython.display import display

canonical = ['Date', 'Symbol' 'Open', 'High', 'Low', 'Last', 'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable', '%Deliverable']

def norm_key(s):
    return s.lower().strip().replace(' ', '').replace('.', '').replace('%', 'pct')

existing = list(df_all.columns)
mapping = {}

for col in existing:
    nk = norm_key(col)
    for can in canonical:
        if nk == norm_key(can):
            mapping[col] = can
            break
        
df = df_all.rename(columns=mapping).copy()

cols_to_drop = []
for c in ['Series', 'Prev Close', 'PrevClose', 'Adj Close', ' Adj_Close', 'Unnamed: 0']:
    matches = [x for x in df.columns if norm_key(x) == norm_key(c)]
    cols_to_drop += matches

cols_to_drop = list(set(cols_to_drop))
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

required = ['Date', 'Symbol']
for r in required:
    if r not in df.columns:
        raise SystemExit(f"Required column missing after cleaning: {r}")
        
df['Symbol'] = df['Symbol'].astype(str).str.strip()

print("Step 4 complete.")
print("Columns now:", list(df.columns))
print("Shape:", df.shape)
display(df.head(6))

Step 4 complete.
Columns now: ['Date', 'Symbol', 'Open', 'High', 'Low', 'Last', 'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble']
Shape: (470384, 13)


,Date,Symbol,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741
5,2007-12-04,MUNDRAPORT,985.00,1056.00,976.0,1049.0,1041.45,1015.39,4849250,4.923867e+14,NaN,1537667.0,0.3171


In [25]:
#conv D
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst = False)
bad_dates = df['Date'].isna().sum()
if bad_dates:
    print(f"Warning: {bad_dates} rows havewrong Date format and will be drop")
df = df.dropna(subset=['Date']).copy()
    
num_candidates = [c for c in ['Open', 'High', 'Low', 'Low', 'Last', 'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable', '%Deliverable']if c in df.columns]

def clean_num(s):
    s2 = s.astype(str).str.replace(',', '', regex=False).str.replace('%', '', regex=False)
    s2 = s2.str.replace(r'^\((.*)\)$', r'-\1', regex = True)
    return pd.to_numeric(s2, errors='coerce')
    
for col in num_candidates:
    df[col]= clean_num(df[col])
    
print("Numeric coercion summary:")
for col in num_candidates:
    n_na = df[col].isna().sum()
    print(f" {col}: {n_na} NaNs / {len(df)} rows")

if 'Close' in df.columns:
    missing_close = df['Close'].isna().sum()
    if missing_close:
        print(f"Note: {missing_close} rows missing close. will  be drop (recommended)")
        df = df.dropna(subset=['Close']).copy()

dup_count = df.duplicated(subset=['Symbol', 'Date']).sum()
if dup_count:
    print(f"Found {dup_count} duplicates (Symbol,Date) rows. Duplicating (keeping last).")
    df = df.sort_values(['Symbol','Date']).drop_duplicates(subset=['Symbol','Date'], keep='last')

print(df.shape)

Numeric coercion summary:
 Open: 0 NaNs / 470384 rows
 High: 0 NaNs / 470384 rows
 Low: 0 NaNs / 470384 rows
 Low: 0 NaNs / 470384 rows
 Last: 0 NaNs / 470384 rows
 Close: 0 NaNs / 470384 rows
 VWAP: 0 NaNs / 470384 rows
 Volume: 0 NaNs / 470384 rows
 Turnover: 0 NaNs / 470384 rows
 Trades: 229696 NaNs / 470384 rows
Found 235192 duplicates (Symbol,Date) rows. Duplicating (keeping last).
(235192, 13)


In [29]:
#S, E
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)

total_rows = len(df)
distinct_tickers = df['Symbol'].nunique()
date_min = df['Date'].min()
date_max = df['Date'].max()
print("Final dataset summary: ")
print(f" Rows: {total_rows}")
print(f"  Distinct tickers: {distinct_tickers}")
print(f"  Date range: {date_min.date()}  ->  {date_max.date()}")

clean_path = os.path.join(output_fl, "nifty50_clean.csv")
df.to_csv(clean_path, index=False)
print(f" Cleaned CSV: {clean_path}")

df.head(200).to_csv(os.path.join(output_fl, "nifty50_clean_preview.csv"), index=False)
types_path = os.path.join(output_fl, "nifty50_column_types.txt")
with open(types_path, 'w', encoding='utf8') as fh:
    for c in df.columns:
        fh.write(f"{c}: {df[c].dtype}\n")
print(f"Preview & types written to output folder: {output_fl}")

display(df.head(6))
print("\nColumn dtypes:")
print(df.dtypes)

Final dataset summary: 
 Rows: 235192
  Distinct tickers: 65
  Date range: 2000-01-03  ->  2021-04-30
 Cleaned CSV: C:\Users\Asus\Desktop\nifty50_output\nifty50_clean.csv
Preview & types written to output folder: C:\Users\Asus\Desktop\nifty50_output


,Date,Symbol,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2012-01-17,ADANIPORTS,137.10,141.00,135.00,140.1,140.00,138.13,1636196,2.260074e+13,18374.0,1004327.0,0.6138
1,2012-01-18,ADANIPORTS,142.00,143.80,138.70,143.0,141.70,141.25,890591,1.257986e+13,15615.0,404925.0,0.4547
2,2012-01-19,ADANIPORTS,144.00,150.55,143.15,149.5,149.40,146.72,1456077,2.136382e+13,31299.0,721545.0,0.4955
3,2012-01-20,ADANIPORTS,151.90,157.60,150.25,155.4,155.40,153.76,1634070,2.512583e+13,23335.0,861145.0,0.5270
4,2012-01-23,ADANIPORTS,155.40,155.40,145.10,146.4,146.75,149.54,1657609,2.478768e+13,12400.0,820653.0,0.4951
5,2012-01-24,ADANIPORTS,147.05,152.90,145.60,149.8,150.05,150.29,1337362,2.009899e+13,15441.0,703939.0,0.5264



Column dtypes:
Date                  datetime64[ns]
Symbol                        object
Open                         float64
High                         float64
Low                          float64
Last                         float64
Close                        float64
VWAP                         float64
Volume                         int64
Turnover                     float64
Trades                       float64
Deliverable Volume           float64
%Deliverble                  float64
dtype: object
